In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys 
import os 

sys.path.append('../../../')
from src.athena import Athena
from src.utils import create_session

## Global

In [12]:
boto3_session = create_session(profile_name='dev', role_arn='arn:aws:iam::722696965592:role/athena-full-access-role')

wait = True

sql_path = "../sql/"

athena = Athena(boto3_session=boto3_session, s3_output=os.getenv('ATHENA_S3_OUTPUT'))
athena

Athena(boto3_session=Session(region_name='us-east-1'), s3_output=s3://sql-case-studies/query_results)

## Create Database & Table

In [13]:
response = athena.create_database(
    database='dannys_diner',
    wait=wait
)

response

Query executed successfully


In [20]:
tables = ['menu', 'sales', 'members']
ddls = {}
for table in tables:
    with open(os.path.join(sql_path, f'{table}_ddl.sql'), 'r') as f:
        ddls[table] = f.read()

    print(ddls[table])
    print('-'*50)
    print('\n')

CREATE EXTERNAL TABLE IF NOT EXISTS dannys_diner.menu (
  product_id INT,
  product_name STRING,
  price DOUBLE
)
COMMENT 'Menu details'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATION 's3://sql-case-studies/dannys_diner/menu/'
TBLPROPERTIES ('has_encrypted_data'='false', 'classification'='parquet', 'skip.header.line.count'='1');

--------------------------------------------------


CREATE EXTERNAL TABLE IF NOT EXISTS dannys_diner.sales (
  customer_id CHAR(1),
  order_date TIMESTAMP,
  product_id INT
)
COMMENT 'Sales line items'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATION 's3://sql-case-studies/dannys_diner/sales/'
TBLPROPERTIES ('has_encrypted_data'='false', 'classification'='parquet', 'skip.header.line.count'='1');

--------------------------------------------------


CREATE EXTERNAL TABLE IF NOT EXISTS dannys_diner.members (
  customer_id CHAR(1),
  join_date TIMESTAMP
)
COMMENT 'Membership details'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATION 's3://sql-case-studies/dannys_d

In [21]:
for ddl in ddls.values():
    response = athena.create_table( database='dannys_diner', query=ddl, wait=wait)
    response

Query executed successfully
Query executed successfully
Query executed successfully


## Drop Database & Table

In [22]:
for table in tables:    
    athena.drop_table(
        database='dannys_diner',
        table=table,
        wait=wait
    )

Query executed successfully
Query executed successfully
Query executed successfully


In [23]:
athena.drop_database(
    database='dannys_diner',
    wait=wait
)

Query executed successfully
